## Use PromptTemplate to create a template for a string prompt
By default, PromptTemplate use Python's str.format syntax for templating

In [89]:
import os
from dotenv import load_dotenv
import openai
#
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [90]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [91]:
model = OpenAI(openai_api_key=api_key,model="gpt-3.5-turbo-instruct",temperature=0.5) #gpt-4-0125-preview

### A simple string based Prompt formatting

In [92]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}")
prompt = prompt_template.format(adjective="stupid",content="Vietnamese")
print("-----------------")
print(prompt_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['adjective', 'content'] template='Tell me a {adjective} joke about {content}'
-----------------
Tell me a stupid joke about Vietnamese
-----------------


Why did the Vietnamese man refuse to eat pho?

Because he was already full of pho-klore!


### ChatPromptTemplate: The prompt to chat model is a list of chat message
Each chat message is associated with content and an additional parameter called role.

For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human, or a system role.

(Mỗi tin nhắn trò chuyện được liên kết với nội dung và một tham số bổ sung gọi là vai trò.

Ví dụ: trong API hoàn thành trò chuyện OpenAI, tin nhắn trò chuyện có thể được liên kết với trợ lý AI, con người hoặc vai trò hệ thống.)

In [93]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI bot. Your name is {name}"),
        ("human","Hello, how are you doing?"),
        ("ai","I'm doing well, thanks!"),
        ("human","{user_input}"),
    ]
)
prompt = chat_template.format_messages(name="Bob",
                                       user_input="What is you name and what are you good at?"
                                      )
print("-----------------")
print(chat_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['name', 'user_input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are a helpful AI bot. Your name is {name}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))]
-----------------
[SystemMessage(content='You are a helpful AI bot. Your name is Bob'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is you name and what are you good at?')]
-----------------

AI: My name is Bob and I am an AI designed to assist and provide helpful information to users. I am programmed to be knowledgeable on a variety of topics and to continuously learn and improve my abilities

### Various ways of formatting System/Human/AI prompts

In [94]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to sound more beat" # Bạn là một trợ lý hữu ích giúp viết lại văn bản của người dùng để nghe có nhịp điệu hơn
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
        AIMessagePromptTemplate.from_template("Give an answer in English and then translate the answer to Vietnamese")
    ]
)
prompt = chat_template.format_messages(text="I don't like eating tasty things") #Tôi không thích ăn đồ ngon.
print("-----------------")
print(chat_template)
print("-----------------")
print(prompt)
response = model.invoke(prompt)
print("-----------------")
print(response)

-----------------
input_variables=['text'] messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more beat"), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Give an answer in English and then translate the answer to Vietnamese'))]
-----------------
[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more beat"), HumanMessage(content="I don't like eating tasty things"), AIMessage(content='Give an answer in English and then translate the answer to Vietnamese')]
-----------------


I'm not a fan of indulging in delicious treats. 
Tôi không thích ăn những thứ ngon miệng.


### Providing a Context along with the Prompt

In [95]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """
# Context: Mô hình ngôn ngữ lớn (LLM) là những mô hình mới nhất được sử dụng trong NLP.
# Hiệu suất vượt trội của chúng so với các mô hình nhỏ hơn đã khiến chúng cực kỳ hữu ích cho các nhà phát triển xây dựng các ứng dụng hỗ trợ NLP. 
# Bạn có thể truy cập các mô hình này thông qua thư viện `transformers` của Hugging Face, qua OpenAI bằng thư viện `openai` và qua Cohere bằng thư viện `cohere`.

In [96]:
print(model.invoke(prompt))

Hugging Face, OpenAI, and Cohere offer LLMs.


### Langchain’s ```FewShotPromptTemplate``` caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.
(FewShotPromptTemplate của Langchain phục vụ cho việc cung cấp nguồn kiến thức đầu vào. Ý tưởng là “đào tạo” mô hình trên một vài ví dụ - chúng tôi gọi đây là few-shot learning - và những ví dụ này được cung cấp cho mô hình trong prompt.)

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.<br>
(Mục tiêu của few-shot prompt templates là chọn động các ví dụ dựa trên thông tin đầu vào, sau đó định dạng các ví dụ trong final prompt để cung cấp cho mô hình.)

<b>Fixed Examples</b><br>
The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production. <br>

The basic components of the template are: 
- examples: A list of dictionary examples to include in the final prompt. 
- example_prompt: converts each example into 1 or more messages through its format_messages method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

In [97]:
from langchain import FewShotPromptTemplate
# create examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do"
    },
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch"
    }
]
# create a eample template
example_templates = """
User: {query}
AI: {answer}
"""
# create a prompt example from above template
example_prompt = PromptTemplate(input_variables=["query","answer"],
                                template=example_templates
                               )
print("-----------------")
print(example_prompt)

-----------------
input_variables=['answer', 'query'] template='\nUser: {query}\nAI: {answer}\n'


In [98]:
# Now break our previous prompt into a prefix and suffix (Bây giờ hãy chia prompt trước đó thành tiền tố và hậu tố)
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the user's questions. 
Here are some examples: 
"""
#Sau đây là đoạn trích từ cuộc trò chuyện với trợ lý AI. 
#Trợ lý này thường có tính châm biếm và hóm hỉnh, đưa ra những câu trả lời sáng tạo và hài hước cho các câu hỏi của người dùng. 
#Dưới đây là một số ví dụ:

# the suffix is our user input and output indicator
suffix = """
User: {query}
"""

In [99]:
# Create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="+++++++++++"
)
query = "Who is the most stupid person in Vietnam?"
print(few_shot_prompt_template)

input_variables=['query'] examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do"}, {'query': 'What time is it?', 'answer': "It's time to get a watch"}] example_prompt=PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\nAI: {answer}\n') suffix='\nUser: {query}\n' example_separator='+++++++++++' prefix="The following are excerpts from conversations with an AI assistant. \nThe assistant is typically sarcastic and witty, producing creative and funny responses to the user's questions. \nHere are some examples: \n"


In [100]:
print(few_shot_prompt_template.format(query=query))

The following are excerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the user's questions. 
Here are some examples: 
+++++++++++
User: How are you?
AI: I can't complain but sometimes I still do
+++++++++++
User: What time is it?
AI: It's time to get a watch
+++++++++++
User: Who is the most stupid person in Vietnam?



In [101]:
# create chain
chain = few_shot_prompt_template | model
response = chain.invoke({"query":query})
print(response)

AI: I'm an AI, I don't have the capability to determine someone's intelligence. But if you're asking for the most foolish decision, I'd say whoever thought it was a good idea to invade Russia in the winter.
